In [2]:
import tensorflow as tf
import numpy as np
import tfgraphviz as tfg
from tensorboard import notebook
from keras.models import Sequential 
from keras.layers import Activation, Dense 
tf.compat.v1.keras.layers.Dense

tensorflow.python.keras.layers.core.Dense

In [3]:
# to make this notebook's output stable across runs
def reset_graph(seed=42):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)
    
def shuffle_batch(X, y, batch_size):
    rnd_idx = np.random.permutation(len(X))
    n_batches = len(X) // batch_size
    for batch_idx in np.array_split(rnd_idx, n_batches):
        X_batch, y_batch = X[batch_idx], y[batch_idx]
        yield X_batch, y_batch

In [4]:
n_inputs = 28*28 # MNIST
n_hidden1 = 300
n_hidden2 = 100
n_outputs = 10

In [5]:
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.mnist.load_data()
X_train = X_train.astype(np.float32).reshape(-1, 28*28) / 255.0
X_test = X_test.astype(np.float32).reshape(-1, 28*28) / 255.0
y_train = y_train.astype(np.int32)
y_test = y_test.astype(np.int32)
X_valid, X_train = X_train[:5000], X_train[5000:]
y_valid, y_train = y_train[:5000], y_train[5000:]

In [6]:
reset_graph()

X = tf.placeholder(tf.float32, shape=(None, n_inputs), name="X")
y = tf.placeholder(tf.int32, shape=(None), name="y")

with tf.name_scope("dnn"):
    hidden1 = tf.layers.dense(X, n_hidden1, name="hidden1",
                              activation=tf.nn.relu)
    hidden2 = tf.layers.dense(hidden1, n_hidden2, name="hidden2",
                              activation=tf.nn.relu)
    logits = tf.layers.dense(hidden2, n_outputs, name="outputs")
    y_proba = tf.nn.softmax(logits)

with tf.name_scope("loss"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)
    loss = tf.reduce_mean(xentropy, name="loss")    

learning_rate = 0.01

with tf.name_scope("train"):
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    training_op = optimizer.minimize(loss)
    
with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

Instructions for updating:
Use keras.layers.dense instead.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [7]:
init = tf.global_variables_initializer()
saver = tf.train.Saver()

In [8]:
n_epochs = 50
batch_size = 50

with tf.Session() as sess:
    init.run()
    for epoch in range(n_epochs):
        for X_batch, y_batch in shuffle_batch(X_train, y_train, batch_size):
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
        acc_train = accuracy.eval(feed_dict={X: X_batch, y: y_batch})
        acc_test = accuracy.eval(feed_dict={X: X_valid, y: y_valid})
        print("Epoch:", epoch, "\tTrain accuracy: {:.2f}%". format(acc_train * 100), 
              "\t\tTest accuracy: {:.3f}%". format(acc_test * 100))
    
    save_path = saver.save(sess, "./my_model_finalv1.ckpt")

Epoch: 0 	Train accuracy: 90.00% 		Test accuracy: 90.280%
Epoch: 1 	Train accuracy: 92.00% 		Test accuracy: 92.520%
Epoch: 2 	Train accuracy: 94.00% 		Test accuracy: 93.720%
Epoch: 3 	Train accuracy: 90.00% 		Test accuracy: 94.200%
Epoch: 4 	Train accuracy: 94.00% 		Test accuracy: 94.720%
Epoch: 5 	Train accuracy: 94.00% 		Test accuracy: 95.120%
Epoch: 6 	Train accuracy: 100.00% 		Test accuracy: 95.480%
Epoch: 7 	Train accuracy: 94.00% 		Test accuracy: 96.120%
Epoch: 8 	Train accuracy: 96.00% 		Test accuracy: 96.200%
Epoch: 9 	Train accuracy: 94.00% 		Test accuracy: 96.500%
Epoch: 10 	Train accuracy: 92.00% 		Test accuracy: 96.540%
Epoch: 11 	Train accuracy: 98.00% 		Test accuracy: 96.720%
Epoch: 12 	Train accuracy: 98.00% 		Test accuracy: 96.820%
Epoch: 13 	Train accuracy: 98.00% 		Test accuracy: 97.040%
Epoch: 14 	Train accuracy: 100.00% 		Test accuracy: 96.960%
Epoch: 15 	Train accuracy: 94.00% 		Test accuracy: 97.160%
Epoch: 16 	Train accuracy: 98.00% 		Test accuracy: 97.260%
Epoch

In [9]:
with tf.Session() as sess:
    saver.restore(sess, "./my_model_finalv1.ckpt") # or better, use save_path
    X_new_scaled = X_test[:20]
    Z = logits.eval(feed_dict={X: X_new_scaled})
    y_pred = np.argmax(Z, axis=1)

Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from ./my_model_finalv1.ckpt


In [10]:
print("Predicted classes:", y_pred)
print("Actual classes:   ", y_test[:20])

Predicted classes: [7 2 1 0 4 1 4 9 5 9 0 6 9 0 1 5 9 7 3 4]
Actual classes:    [7 2 1 0 4 1 4 9 5 9 0 6 9 0 1 5 9 7 3 4]


In [11]:
g = tfg.board(tf.get_default_graph())
g.view()

'G.gv.pdf'

In [12]:
# Load the TensorBoard notebook extension
%load_ext tensorboard
#%reload_ext tensorboard

In [13]:
#%tensorboard --logdir ./
%tensorboard --logdir tf_logs
#!kill 14672

Reusing TensorBoard on port 6006 (pid 3080), started 7:55:32 ago. (Use '!kill 3080' to kill it.)

## Resposta:
Analisando o exemplo MLPWithTensorFlowLowLevelAPI.ipynb utilizando objetos da função neuron_layer() e utilizando objetos da classe dense() e não foi observada nenhuma diferença significativa de performance.